In [1]:
import pandas as pd
import numpy as np
from pyproj import Proj, transform #위도 경도 변환  proj 설치
import re
import folium #folium 설치
import matplotlib.pyplot as plt
import json #지도 그릴때 사용 

In [2]:
data = pd.read_csv('./서울특별시 휴게음식점 인허가 정보.csv', encoding = 'cp949') 
print(data.info())
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107486 entries, 0 to 107485
Data columns (total 44 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   개방자치단체코드   107486 non-null  int64  
 1   관리번호       107486 non-null  object 
 2   인허가일자      107486 non-null  int64  
 3   인허가취소일자    0 non-null       float64
 4   영업상태코드     107486 non-null  int64  
 5   영업상태명      107486 non-null  object 
 6   상세영업상태코드   107486 non-null  int64  
 7   상세영업상태명    107486 non-null  object 
 8   폐업일자       70946 non-null   float64
 9   휴업시작일자     0 non-null       float64
 10  휴업종료일자     0 non-null       float64
 11  재개업일자      0 non-null       float64
 12  전화번호       57186 non-null   object 
 13  소재지면적      99169 non-null   float64
 14  소재지우편번호    107266 non-null  float64
 15  지번주소       107268 non-null  object 
 16  도로명주소      65834 non-null   object 
 17  도로명우편번호    65170 non-null   float64
 18  사업장명       107486 non-null  object 
 19  최종수정일자     107486 non-n

c:\users\023\anaconda3\envs\machinelearning\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (36,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
list_ = ['단란주점영업', '유흥주점영업','일반음식점','제과점영업','관광음식점']

In [4]:
for a in list_:
    df = pd.read_csv('./서울특별시 {} 인허가 정보.csv'.format(a), encoding = 'cp949',index_col=0) 
    data = pd.concat([data, df], ignore_index=True)
    print('concat')

concat
concat


c:\users\023\anaconda3\envs\machinelearning\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (25,28,29,30,36,39,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


concat
concat
concat


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590908 entries, 0 to 590907
Data columns (total 79 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   개방자치단체코드    107486 non-null  float64
 1   관리번호        590908 non-null  object 
 2   인허가일자       590908 non-null  int64  
 3   인허가취소일자     8 non-null       float64
 4   영업상태코드      590908 non-null  int64  
 5   영업상태명       590908 non-null  object 
 6   상세영업상태코드    590908 non-null  int64  
 7   상세영업상태명     590908 non-null  object 
 8   폐업일자        423101 non-null  float64
 9   휴업시작일자      0 non-null       float64
 10  휴업종료일자      0 non-null       float64
 11  재개업일자       0 non-null       float64
 12  전화번호        404063 non-null  object 
 13  소재지면적       544178 non-null  float64
 14  소재지우편번호     590322 non-null  float64
 15  지번주소        590341 non-null  object 
 16  도로명주소       283755 non-null  object 
 17  도로명우편번호     279259 non-null  float64
 18  사업장명        590907 non-null  object 
 19  최종

In [6]:
new_data = data[['상세영업상태명','인허가일자', '폐업일자','지번주소','도로명주소','업태구분명','좌표정보(X)','좌표정보(Y)']]
new_data.reset_index(drop=True, inplace=True)
new_data.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590908 entries, 0 to 590907
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   상세영업상태명  590908 non-null  object 
 1   인허가일자    590908 non-null  int64  
 2   폐업일자     423101 non-null  float64
 3   지번주소     590341 non-null  object 
 4   도로명주소    283755 non-null  object 
 5   업태구분명    590668 non-null  object 
 6   좌표정보(X)  560670 non-null  float64
 7   좌표정보(Y)  560670 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 36.1+ MB


In [7]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590908 entries, 0 to 590907
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   상세영업상태명  590908 non-null  object 
 1   인허가일자    590908 non-null  int64  
 2   폐업일자     423101 non-null  float64
 3   지번주소     590341 non-null  object 
 4   도로명주소    283755 non-null  object 
 5   업태구분명    590668 non-null  object 
 6   좌표정보(X)  560670 non-null  float64
 7   좌표정보(Y)  560670 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 36.1+ MB


In [8]:
new_data.dropna(subset=['좌표정보(X)'],how='any',inplace=True)
new_data.dropna(subset=['좌표정보(Y)'],how='any',inplace=True)

c:\users\023\anaconda3\envs\machinelearning\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\users\023\anaconda3\envs\machinelearning\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
new_data.info()
new_data.to_csv('./cleaned_data.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 560670 entries, 0 to 590907
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   상세영업상태명  560670 non-null  object 
 1   인허가일자    560670 non-null  int64  
 2   폐업일자     394840 non-null  float64
 3   지번주소     560112 non-null  object 
 4   도로명주소    282756 non-null  object 
 5   업태구분명    560446 non-null  object 
 6   좌표정보(X)  560670 non-null  float64
 7   좌표정보(Y)  560670 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 38.5+ MB


In [10]:
#위도,경도 변환
proj_UTMK = Proj(init='epsg:2097')
proj_WGS84 = Proj(init='epsg:4326')

c:\users\023\anaconda3\envs\machinelearning\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
c:\users\023\anaconda3\envs\machinelearning\lib\site-packages\pyproj\crs\crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
c:\users\023\anaconda3\envs\machinelearning\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the prefe

In [11]:
x1,y1=int(new_data[['좌표정보(X)']].iloc[0]),int(new_data[['좌표정보(Y)']].iloc[0])         #198867.702590612 ,451824.281125555
x2,y2 = transform(proj_UTMK,proj_WGS84,x1,y1)
print(x2,y2)

127.01507842764127 37.61086698615127


c:\users\023\anaconda3\envs\machinelearning\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  


In [12]:
x=[]
y=[]
for i in range(len(new_data[['좌표정보(X)']])):
    x1,y1=int(new_data[['좌표정보(X)']].iloc[i]),int(new_data[['좌표정보(Y)']].iloc[i])
    x2,y2 = transform(proj_UTMK,proj_WGS84,x1,y1)
    x.append(x2)
    y.append(y2)
    if i%10==0:
       print('.',end='')
new_data['X']=x
new_data['Y']=y
new_data.to_csv('./loc_add_data.csv')

c:\users\023\anaconda3\envs\machinelearning\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  """


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

c:\users\023\anaconda3\envs\machinelearning\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
c:\users\023\anaconda3\envs\machinelearning\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
